In [1]:
import sys
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
# from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint, Callback
from keras_contrib.layers import CRF
from numpy import newaxis
import sklearn
import subprocess
import fastText
import pickle

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]


def createMatrices(sentences, label2Idx, case2Idx,char2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}

        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []
        caseIndices = []
        charIndices = []
        labelIndices = []
        
        for word,char,label in sentence:  
            charIdx = []
            for x in char:
                if x in char2Idx.keys():
                    charIdx.append(char2Idx[x])
                else:
                    charIdx.append(char2Idx['UNKNOWN'])
            #Get the label and map to int            
            wordIndices.append(word)
            caseIndices.append(getCasing(word, case2Idx))
            charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, charIndices, labelIndices]) 
        
    return dataset

def padding(Sentences):
    maxlen = 52
    for sentence in Sentences:
        char = sentence[2]
        for x in char:
            maxlen = max(maxlen,len(x))
    for i,sentence in enumerate(Sentences):
        Sentences[i][2] = pad_sequences(Sentences[i][2],52,padding='post')
    return Sentences



In [5]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [6]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [32]:
def get_sentences_germeval(path):
    sentences = []
    with open(path, 'r', encoding = 'UTF-8') as f:
        sentence = []
        for line in f:
            
            line = line.strip()
            
            # append sentence
            if len(line) == 0:
                if len(sentence):
                    sentences.append(sentence)
                sentence = []
                continue
            
            # get sentence tokens
            splits = line.split()
            if splits[0] == '#':
                continue
            temp = [splits[1],splits[2]]
            sentence.append(temp)
        
        # append last
        if len(sentence):
            sentences.append(sentence)    
    return sentences

In [33]:
# preproecessing data from Conll
def get_sentences_conll(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [34]:
trainSentences = get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [35]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-OTH'], ['gebaut', 'O'], ['.', 'O']]


In [36]:
labelSet = set()
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)

In [37]:
print(len(labelSet))

25


In [38]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [39]:
print(label2Idx)

{'B-PERpart': 0, 'B-LOCderiv': 16, 'B-PER': 8, 'B-OTHderiv': 9, 'I-OTHpart': 10, 'I-OTH': 11, 'I-OTHderiv': 12, 'B-PERderiv': 13, 'I-PERderiv': 14, 'B-LOCpart': 1, 'I-ORGpart': 2, 'B-LOC': 15, 'B-OTHpart': 3, 'I-ORGderiv': 18, 'B-ORGpart': 4, 'B-ORGderiv': 19, 'B-ORG': 20, 'O': 5, 'I-ORG': 6, 'I-PER': 17, 'I-LOC': 21, 'B-OTH': 22, 'I-LOCpart': 23, 'I-PERpart': 7, 'I-LOCderiv': 24}


In [40]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [41]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'other': 4, 'contains_digit': 6, 'allUpper': 2, 'mainly_numeric': 5, 'allLower': 1, 'PADDING_TOKEN': 7, 'numeric': 0, 'initialUpper': 3}


In [42]:
print(type(trainSentences))

<class 'list'>


In [43]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [44]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'R': 106, '1': 0, 'Е': 240, '~': 2, 'ъ': 313, 'ế': 3, '¹': 4, '£': 5, 'I': 6, 'ý': 7, 'î': 8, 'Á': 9, '?': 10, 'П': 279, '³': 12, '佐': 268, '鷹': 13, 'κ': 14, 'Л': 63, '南': 15, 'ô': 16, '‘': 17, 'İ': 19, 'ž': 35, '_': 22, 'Ġ': 293, 'С': 222, 'ā': 23, 'ç': 1, '’': 24, '€': 25, 'j': 28, 'з': 29, 'ó': 30, '≤': 31, 'Ż': 32, 'G': 247, '`': 33, 'ħ': 11, 'Ş': 38, '별': 40, 'š': 39, '\x94': 41, 'ō': 42, '#': 218, 'ń': 43, '°': 217, '公': 44, 'υ': 45, '⊃': 46, 'ب': 47, 'i': 48, 'à': 51, '}': 50, 'н': 52, '×': 53, 'ö': 55, '章': 56, '‐': 57, 'û': 58, 'M': 59, 'ś': 62, ':': 281, '\u200e': 114, 'ú': 64, 'р': 65, '–': 286, 'ŏ': 296, 'k': 67, 'ę': 181, 'ē': 70, '守': 75, 'a': 71, 'P': 68, 'ـ': 74, 'd': 76, 'τ': 69, 'ū': 78, 'М': 282, 'д': 80, '太': 321, 'ê': 82, 'á': 256, 'ε': 117, 'È': 90, 'ř': 73, 'z': 86, 'í': 177, '$': 314, 'ḳ': 91, 'L': 180, 'Q': 92, '算': 310, 'D': 93, 'ς': 94, ']': 95, '대': 96, 'λ': 97, '台': 99, '%': 87, 'ĩ': 100, 'ë': 101, 'В': 102, 'ρ': 104, 'c': 124, 'о': 105, '<': 18, 'е': 108,

In [45]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [21]:
ft = fastText.load_model("../embeddings/wiki.de.bin")
# ft = fastText.load_model("../embeddings/cc.de.300.bin")

nb_embedding_dims = ft.get_dimension()

In [22]:
print(nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [46]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [47]:
def createBatches(dataset):
    l = []
    for i in dataset:
        l.append(len(i))
    l = set(l)
    print(len(l))
    batches = []
    batch_len = []
    z = 0
    for i in l:
        temp = []
        for batch in dataset:
            if len(batch) == i:
                temp.append(batch)
                z += 1
        batches.append(temp)
#         batch_len.append(z)
    return batches

In [48]:
train_batches = createBatches(trainSentences)
dev_batches = createBatches(devSentences)
test_batches = createBatches(testSentences)

51
45
47


In [49]:
batches = train_batches
print(len(batches))
print(len(batches[0]))
print(batches[0])


51
3
[[['Alles', 'O'], ['richtig', 'O'], ['.', 'O']], [['Farben', 'O'], ['eingeführt', 'O'], ['.', 'O']], [['Material', 'O'], ['gewinnen', 'O'], ['.', 'O']]]


In [50]:
def generator(batches: 'list of training/dev sentences- batches already created'):
    global line_number
    
    while True:
        for batch in batches:
            word_embeddings = []
            case_embeddings = []
            char_embeddings = []

            output_labels = []
            for index in range(len(batch)): # batches made according to the size of the sentences. len(batch) gives the size of current batch
                sentence = batch[index]
    #             print(sentence)
                temp_casing = []
                temp_char=[]
                temp_word=[]
                temp_output=[]
                for word in sentence:
                    word, label = word
                    casing =getCasing(word, case2Idx)
                    temp_casing.append(casing)
                    temp_char2=[]
                    for char in word:
                        if char in char2Idx.keys():
                            temp_char2.append(char2Idx[char])
                        else:
                            temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                    temp_char2 = np.array(temp_char2)
                    temp_char.append(temp_char2)
                    word_vector = ft.get_word_vector(word.lower())
                    # word_vector = ft.get_word_vector(word)
                    temp_word.append(word_vector)
                    temp_output.append(label2Idx[label])
                temp_char = pad_sequences(temp_char, 52)
                word_embeddings.append(temp_word)
                case_embeddings.append(temp_casing)
                char_embeddings.append(temp_char)
                temp_output = to_categorical(temp_output, 25)
                output_labels.append(temp_output)
    #             output_labels = to_categorical()
    #             output_labels = np.array(output_labels)
    #             output_labels = output_labels[...,newaxis]

    #             print(np.array(word_embeddings).shape)
    #             print(np.array(case_embeddings).shape)
    #             print(np.array(char_embeddings).shape)
    #             print(output_labels.shape)
    #             print("******************\n\n")
            yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

def get_label_from_categorical(a):
    labels = []
    for label in a:
        label = np.ndarray.tolist(label)
        label = np.argmax(label)
        labels.append(label)
    return(labels)

def predict_batches(batch):
    steps = 0
    true_labels = []
    pred_labels = []
    for input_data, output_data in generator(batch):
        pred_labels_batch = model.predict(input_data)
        for s in pred_labels_batch:
            pred_labels.append(get_label_from_categorical(s))
        for s in output_data:
            true_labels.append(get_label_from_categorical(s))
        steps += 1
        if steps == len(batch):
            break
    return(true_labels, pred_labels)

In [51]:
idx2Label = {v: k for k, v in label2Idx.items()}

In [29]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [52]:
def f1_score(y_true, y_pred):
    true_labels = []
    pred_labels = []
    for s in y_pred:
        pred_labels.append(get_label_from_categorical(s))
    for s in y_true:
        true_labels.append(get_label_from_categorical(s))
    p, r, f = compute_f1(y_pred, y_true, idx2Label)
    return r

def get_model():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    kernel_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in kernel_sizes:
        conv = TimeDistributed(Conv1D(
                             kernel_size=sz,
                             filters=32,
                             padding="same",
                             activation="relu",
                             strides=1))(embed_char_out)
        conv = TimeDistributed(MaxPooling1D(52))(conv)
        conv = TimeDistributed(Flatten())(conv)
        conv_blocks.append(conv)
    output = concatenate([words_input, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
    output = TimeDistributed(Dense(len(label2Idx)))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

class F1History(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.f1_scores = []
        self.max_f1 = 0

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs.get('val_acc'))
        true_labels, pred_labels = predict_batches(dev_batches)
        pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
        self.f1_scores.append(f1)
        if epoch > 30 and f1 > self.max_f1:
            print("\nNew maximum F1 score: " + str(f1) + " (before: " + str(self.max_f1) + ") Saving to " + tmp_model_filename)
            self.max_f1 = f1
            model.save(tmp_model_filename)

In [53]:
tmp_model_filename = 'tmp_generator_NER_best.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = F1History()
model = get_model()
model.fit_generator(
    generator(train_batches), 
    epochs = 80, steps_per_epoch = len(train_batches), 
    validation_data = generator(dev_batches), validation_steps = len(dev_batches), 
    callbacks = [history]
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10528       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_11 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_14 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

KeyboardInterrupt: 

In [ ]:
print(history.acc)
print(history.f1)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = predict_batches(test_batches)
print(compute_f1(pred_labels, true_labels, idx2Label))

# Experiments

In [ ]:
f = open('results_wiki.txt', 'w')
for run_i in range(13):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_best.' + str(run_i) + '.h5'
    # tmp_model_filename = 'tmp_generator_NER_best.h5'
    # checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
    history = F1History()
    
    model = get_model()
    model.fit_generator(
        generator(train_batches), 
        epochs = 80, steps_per_epoch = len(train_batches), 
        validation_data = generator(dev_batches), validation_steps = len(dev_batches), 
        callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    true_labels, pred_labels = predict_batches(test_batches)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_generator_NER_best.0.h5', 'final_model_germeval.h5')
with open("final_model_germeval.indexes", "w") as f:
    json.dump([idx2Label, label2Idx, char2Idx, case2Idx], f)